In [ ]:
from icalendar import Calendar, Event
from datetime import datetime, timedelta
import pytz

In [ ]:
# Imports needed for API
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from datetime import datetime
import os

In [ ]:
# Authentication and API set up. Refer to README.md for more information.
SCOPES = ['https://www.googleapis.com/auth/calendar']
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

In [ ]:
cal = Calendar()
cal.add('prodid', '-//Semester Schedule//mxm.dk//')
cal.add('version', '2.0')

# Set timezone
tz = pytz.timezone('US/Eastern')

# Define semester range
start_date = datetime(2025, 8, 25)  
end_date = datetime(2025, 12, 7)

# class_times = [
#     (10, 0, 13, 0) 
# ]
class_start = [9, 30]
class_end = [10,40]

project_time_start = [8,0]
project_time_end = [10,0]

class_names = ['class_list']
descriptions = [
    "Topic 1",
    "Topic 2",
    "Topic 3",
    "Topic 4"
]

description_index = 0
current_date = start_date

In [ ]:
while current_date <= end_date:
    if current_date.weekday() in [0, 2, 4, 6]:  

        # This part is for required events
        event = Event()
        class_name = class_names[0]
        event.add('summary', class_name)
        event.add('dtstart', tz.localize(datetime(current_date.year, current_date.month, current_date.day, class_start[0], class_start[1])))
        event.add('dtend', tz.localize(datetime(current_date.year, current_date.month, current_date.day, class_end[0], class_end[1])))
        event.add('location', "Engineering Building")
        
        


        event.add('transp', 'OPAQUE')
        cal.add_component(event)

        
        # This part is for optional all day events
        optional_event = Event()
        optional_event.add('summary', "Project")
        # date ensures there is no time limit set
        optional_event.add('dtstart', tz.localize(datetime(current_date.year, current_date.month, current_date.day,0,0)))
        optional_event.add('dtend', tz.localize(datetime(current_date.year, current_date.month, current_date.day ,0,0)) + timedelta(days=1))
        optional_event.add('location', "https://www.google.com/")
        
        # This makes it optional
        optional_event.add('transp', 'TRANSPARENT')  
        cal.add_component(optional_event)

        # This part is for google calendar
        
        g_event = {
            'summary': str(event.get('summary')),
            'location': str(event.get('location')),
            'description': str(event.get('description')),
            'start': {
                'dateTime': event.get('dtstart').dt.isoformat(),
                'timeZone': 'US/Eastern',
            },
            'end': {
                'dateTime': event.get('dtend').dt.isoformat(),
                'timeZone': 'US/Eastern',
            },
        }
        
        service.events().insert(calendarId='primary', body=g_event).execute()

        g_event = {
            'summary': str(optional_event.get('summary')),
            'location': str(optional_event.get('location')),
            'description': str(optional_event.get('description')),
            'start': {
                'dateTime': optional_event.get('dtstart').dt.isoformat(),
                'timeZone': 'US/Eastern',
            },
            'end': {
                'dateTime': optional_event.get('dtend').dt.isoformat(),
                'timeZone': 'US/Eastern',
            },
            'transparency': 'transparent'
        }

        
        service.events().insert(calendarId='primary', body=g_event).execute()


    current_date += timedelta(days=1)
